In [ ]:
%tensorflow_version 1.x

In [ ]:
# 구글 드라이브와 연동하는 방법 입니다. 
# Google File Drive Stream 접근을 허용해야 합니다.
from google.colab import auth
auth.authenticate_user()

# 구글 드라이브 mount 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 1. 홈 디렉토리에 datasets 폴더 생성 후, 그 안에 cats_and_dogs_small 폴더를 생성 합니다.
# 2. cats_and_dogs_small 안에 train, test, validation 폴더가 위치 하도록 압축을 풉니다.
cd ./cats_and_dogs_small

In [ ]:
!ls

In [ ]:
!!pwd

In [1]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

base_dir = './cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
test_dir  = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 224 × 224 크기로 바꿉니다
        target_size=(224, 224),
        batch_size=4,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=4,
        class_mode='categorical')


Using TensorFlow backend.


Found 121 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [2]:
for data_batch, labels_batch in train_generator:
    print ('Batch size:', data_batch.shape)
    print ('Label size:', labels_batch.shape)
    break

Batch size: (4, 224, 224, 3)
Label size: (4, 2)


In [3]:
print (train_generator.class_indices)

{'cats': 0, 'dogs': 1}


In [ ]:
#  This code cell shows how to utilize VGG16 model by combining Dense layer at the end of the network 
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import layers
from keras import models

VGGNet = VGG16()
VGGNet.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [ ]:
# We will not update VGG pre-trained model, only added Dense layers will be trained from the scratch
for layer in VGGNet.layers:
  layer.trainable = False
  
Feature_Flatten = VGGNet.get_layer('flatten').output

dense = Dense(256, name='encoding', activation='relu',)(Feature_Flatten)
predictions = Dense(2, activation='softmax', name="prediction")(dense)

New_VGGmodel = Model(inputs=VGGNet.input, outputs=predictions)
New_VGGmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

New_VGGmodel.summary()

In [ ]:
config = tf.ConfigProto()

config.gpu_options.allow_growth = True

sess = tf.Session(config=config)


In [22]:
history = New_VGGmodel.fit_generator(
    train_generator,
#     validation_data=validation_generator, 
#     validation_steps=5,
    # 20장 / batch_size 4 = 5
    steps_per_epoch=15,
    # 60장 / batch_size 4 = 15
    epochs=5
    # 전체 반복 횟수 입니다.
)

Epoch 1/5


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1_1/convolution}}]]
	 [[Mean_1/_489]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
print (train_generator.class_indis)

In [ ]:
pip install flickrapi

In [ ]:
from flickrapi import FlickrAPI

FLICKER_KEY = '505e8c2b05e8b5aa51766a43fc4bc7a7'
FLICKER_SECRET = '46b3ef2f43b04ac6'

flickr = FlickrAPI(FLICKER_KEY, FLICKER_SECRET, format='parsed-json')
extras='url_s'
cats = flickr.photos.search(text='puppy', per_page=5, extras=extras)
photos = cats['photos']

In [ ]:
photos

In [ ]:
# import the necessary packages
import numpy as np
import urllib
import cv2

def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

In [ ]:
from google.colab.patches import cv2_imshow

image_original_url = photos['photo'][1]['url_s']

image_temp = url_to_image(image_original_url)
cv2_imshow(image_temp)

In [ ]:
# 4D-tensor(B, H, W, C) -> (1, 224, 224, 3)
image_temp.shape

In [ ]:
resized_image = cv2.resize(image_temp, (224, 224))
print (resized_image.shape)

In [ ]:
resized_image = np.expand_dims(resized_image, axis=0)
print (resized_image.shape)

In [ ]:
result = New_VGGmodel.predict(resized_image)

In [ ]:
result

In [ ]:
np.argmax(result)

In [ ]:
!pwd

In [ ]:
New_VGGmodel.save('newvgg_catsdogs.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()